# Entrenando modelo para reconocer palabras

### Obteniendo los comandos iniciales

In [ ]:
#----------------------------------------------------------------------------------
#--------------------------------
#   Speech data aquisition
#----------------------------------------------------------------------------------
#--------------------------------
import time
import random
import numpy as np
import sounddevice as sd
import pickle
from datetime import datetime
# Experiment configuration
conditions = [('Escuela', 1), ('Trabajo', 2), ('Casa', 3), ('Boliche', 4), ('Bar', 5)]
n_trials = 30
fixation_cross_time = 2
preparation_time = 0.3
training_time = 2
rest_time = 1
trials = n_trials*conditions
random.shuffle(trials)
fs=44100    
# Data aquisition
data = []
for i,t in enumerate(trials):
    # Fixation cross
    print ("*********")    
    time.sleep(fixation_cross_time)    
    
    # Preparation time
    print (t[0])
    time.sleep(preparation_time)
    # Task    
    recording = sd.rec(training_time * fs, samplerate=fs, 
channels=2,dtype='float64')    
    sd.wait()
    data.append((t[0], t[1], recording))
    # Rest time
    print(f"----Descansa---- {len(trials) - i - 1} left. ")
    time.sleep(rest_time)
# Play records
for t in data:
    sd.play(t[2], fs)
    sd.wait()
# Save data
outputFile = open('JORGE.obj', 'wb')
pickle.dump(data, outputFile)
outputFile.close()
#----------------------------------------------------------------------------------
#--------------------------------
#   End of file
#----------------------------------------------------------------------------------

### obtener la matriz de features

In [ ]:
import pickle
import numpy as np
from scipy import stats
from scipy import signal
import matplotlib.pyplot as plt
import sounddevice as sd
from python_speech_features import mfcc

def process_voice_commands(file_name):
    #----------------------------------------------------------------------------------
    #--------------------------------
    #   Speech data processing
    #----------------------------------------------------------------------------------
    #--------------------------------



    # Load audio records
    fs = 44100
    inputFile = open(file_name, 'rb')
    data = pickle.load(inputFile)
    n_trials = len(data)
    # Plot one record
    t = 5
    dt = 1/fs
    t_size = data[t][2].shape[0]
    x = np.arange(0, t_size*dt, dt)
    figure, axis = plt.subplots(2, 1)
    axis[0].plot(x, data[t][2][:,0])
    axis[0].set_title("Canal 1")
    axis[1].plot(x, data[t][2][:,1])
    axis[1].set_title("Canal 2")
    plt.show()
    # Filter signals
    filt = signal.iirfilter(4, [10, 15000], rs=60, btype='band',
                           analog=False, ftype='cheby2', fs=fs,
                           output='ba')
    filtered = []
    for tr in data:
        ff1 = signal.filtfilt(filt[0], filt[1], tr[2][:,0], method='gust')
        ff2 = signal.filtfilt(filt[0], filt[1], tr[2][:,1], method='gust')
        filtered.append(np.column_stack((ff1, ff2)))
    figure, axis = plt.subplots(2, 1)
    axis[0].plot(x, filtered[t][:,0])
    axis[0].set_title("Canal 1")
    axis[1].plot(x, filtered[t][:,1])
    axis[1].set_title("Canal 2")
    plt.show()
    sd.play(filtered[t], fs)
    sd.wait()
    sd.play(data[t][2], fs)
    sd.wait()
    # Calculate MFCC features
    mfcc_feat = mfcc(filtered[t], fs, nfft = 2048)
    plt.matshow(mfcc_feat.T)
    plt.show()
    features = []
    for tr in data:
        mfcc_feat = mfcc(tr[2], fs, nfft = 2048)
        features.append(mfcc_feat.flatten())
    # Build x and y arrays
    x = np.array(features)
    y = [row[1] for row in data]
    
    y = np.array(y)
    #return x and y
    return x, y
    
    #----------------------------------------------------------------------------------
    #--------------------------------
    #   End of file
    #----------------------------------------------------------------------------------
    #--------------------------------

### Obteniendo modelo a usar

In [1]:
from sklearn import svm

KeyboardInterrupt: 

In [5]:
def getSVC():
    return svm.SVC(kernel='linear')
def getRad():
    return svm.SVC(kernel='rbf')

In [6]:
modelos = {
    "SVM lineal": getSVC,
    "SVM de base radial": getRad,
}

## Obteniendo features del audio 

In [9]:
print(f"USANDO COMANDOS DE: JORGITO")
file_name = "JORGE.obj"
x, y = process_voice_commands(file_name)
model_string = "SVM lineal"
model = modelos[model_string]
print(f"USANDO MODELO: {modelo_string}")
clas_results = cross_validate(model, x, y, cv = 5, scoring = ('accuracy', 'recall_micro', 'precision_micro'))
print('Acc: ', clas_results['test_accuracy'].sum()/5)
print('Recall: ', clas_results['test_recall_micro'].sum()/5)
print('Precision: ', clas_results['test_precision_micro'].sum()/5)
print()
print()

<function getSVC at 0x129dd4820>


In [ ]:
svc = model()
svc.fit(x,y)

# Planificador de rutas

In [1]:
#Instalando librerías de open street map y de simpleAI
!pip install osmnx simpleai

In [2]:
import osmnx as ox
from simpleai.search import breadth_first, depth_first, SearchProblem, iterative_limited_depth_first, uniform_cost


class FindRoute(SearchProblem):
    def __init__(self, goal, mapa, initial_state=(1377048890), data_cost="length"):
        self.goal = goal
        self.data_cost = data_cost
        self.__G = mapa
        super().__init__(initial_state)

    def actions(self, state):
        act = []
        (current_node) = state
        for element in self.__G.out_edges(current_node, data=self.data_cost):
            #STRUCTURE: (FROM, TO, COST)
            act.append(element)
        return act

    def result(self, state, action):
        (node_from, node_to, action_cost) = action
        return (node_to)
    
    def cost(self, state, action, state2):
        (node_from, node_to, action_cost) = action
        return action_cost

    def is_goal(self, state):
        #It finished when it arrives to the goal node
        (curr_node) = state
        return curr_node == self.goal

class RoutePlanner:
  def __init__(self, algorithm = uniform_cost):
    self.__city = 'Mexico, Guadalajara'
    self.__map = ox.graph_from_place(self.__city, network_type='drive')
    self.__starting_node = 1377048890
    self.__search_algorithm = algorithm

    self.__destinations = {
        "pizza": 8359511705,
        "escuela": 1699102733,
        "hospital": 1343795278,
        "cine": 9795883739,
        "restaurante": 1478411793
    }

  def get_route_to_destination(self, destination, color="r"):
    if destination not in self.__destinations:
      print("No existe el destino ingresado.")
      return

    destination_idx = self.__destinations[destination]
    router = FindRoute(destination_idx, mapa = self.__map, initial_state = (self.__starting_node), data_cost="length")

    route_nodes = []

    result = self.__search_algorithm(router, True)

    if not result:
      print("NO HUBO CAMINO")
    else:
      for action,state in result.path():
        if action:
            (node_from, node_to, action_cost) = action
            #total_cost += action_cost
        (node) = state
        route_nodes.append(node)

      ox.plot.plot_graph_route(self.__map, route_nodes)
